In [1]:
# Youtbue downloader
!pip install youtube-dl
# install dependencies: 
!pip install pyyaml==5.1 'pycocotools>=2.0.1'
!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
# opencv is pre-installed on colab
# install detectron2: (Colab has CUDA 10.1 + torch 1.6)
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
assert torch.__version__.startswith("1.6")
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
1.6.0+cu101 True
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html


In [16]:
import glob
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from datetime import datetime


In [3]:
def download_video(url, download_location):
  os.system(f"youtube-dl {url} -f 22 -o {download_location}")

In [4]:
def extract_frames(video_path, frames_location, period=1):
  os.system(f"ffmpeg -i {video_path} -r {period} {frames_location}")

In [5]:
cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)
# outputs = predictor(im)

In [19]:
def filter_images(images):
  for img in images:
    im = cv2.imread(img)
    outputs = predictor(im)
    # If person in detections or no detections => delete the image
    if (0 in outputs["instances"].pred_classes) or (not outputs["instances"].pred_classes.size()[0]):
      print(f"removing {img}")
      os.system("rm -f "+img)


In [33]:
!mkdir /content/temp1

In [34]:
!mkdir /content/temp2

In [35]:
# list_of_interior_videos = [
# 'https://www.youtube.com/watch?v=gT8XOrR_Ufo',
# 'https://www.youtube.com/watch?v=lGG32t7uzzI',
# 'https://www.youtube.com/watch?v=K42AnnVQvLk',
# 'https://www.youtube.com/watch?v=91QiEs6XQSw',
# 'https://www.youtube.com/watch?v=QmjBpxa3mw8',
# 'https://www.youtube.com/watch?v=u6lEoDcKKG4',
# 'https://www.youtube.com/watch?v=HHAO19IOlws',
# 'https://www.youtube.com/watch?v=1vnHnLXiQOo',
# 'https://www.youtube.com/watch?v=jBFwkYkN9EY',
# 'https://www.youtube.com/watch?v=BjxE72IN6QU',
# 'https://www.youtube.com/watch?v=R9iBFZC4mw4',
# 'https://www.youtube.com/watch?v=LbeG6FMvK1s',
# 'https://www.youtube.com/watch?v=XXqfB34mj2k',
# ]

list_of_interior_videos = [
'https://www.youtube.com/watch?v=qm70dYZgIew',
'https://www.youtube.com/watch?v=NmLJ4ibKBdM',
'https://www.youtube.com/watch?v=pBJprhsBUrg',
'https://www.youtube.com/watch?v=_U46eC3e9YQ',
'https://www.youtube.com/watch?v=5ZZitB_FFE8',
'https://www.youtube.com/watch?v=iMP3Zn2X9sg',
'https://www.youtube.com/watch?v=1hK9cKLqI4o',
'https://www.youtube.com/watch?v=JAcgj8FwwFA',
'https://www.youtube.com/watch?v=CEkMkANMmdA',
'https://www.youtube.com/watch?v=Ry4CH35P3bs',
]
interior_data_location = "/content/drive/My Drive/computer_vision/capstone_dataset/inputs/interior_input_images"
frames_location = "/content/temp1/"
download_location = "/content/temp2/"

In [36]:
def prepare_dataset(list_of_interior_videos, frames_location, interior_data_location, download_location='/content/temp2/'):
  count1 = 1
  video_file_names = []
  for video_url in list_of_interior_videos:
    print(f"Downloading {video_url}")
    target = f"{download_location}video{count1}.mp4"
    video_file_names.append(target)
    download_video(video_url, target)
    print(f"Downloaded {target}")
    count1+=1

  for video_file_name in video_file_names: 
    timestamp = datetime.timestamp(datetime.now()) # current date and time
    print(f"Started extracting frames from {video_file_name}")
    frames_location_final = f"{frames_location}img{timestamp}%3d.jpg"
    extract_frames(video_file_name, frames_location_final, period=1)
    print(f"Ended extracting frames from {video_file_name}")
  
  images = glob.glob(f"{frames_location}*.jpg")
  print(f"Before filtering number of images - {len(images)}")
  filter_images(images) ## Detect person or no objects and delete those images - Through Detectron2 using maskrcnn
  images = glob.glob(f"{frames_location}*.jpg")
  print(f"After filtering number of images - {len(images)}")

In [37]:
prepare_dataset(list_of_interior_videos, frames_location, interior_data_location, download_location)

Downloaded /content/video1.mp4
Downloaded /content/video2.mp4
Downloaded /content/video3.mp4
Downloaded /content/video4.mp4
Downloaded /content/video5.mp4
Downloaded /content/video6.mp4
Downloaded /content/video7.mp4
Downloaded /content/video8.mp4
Downloaded /content/video9.mp4
Downloaded /content/video10.mp4
Started extracting frames from /content/video1.mp4
Ended extracting frames from /content/video1.mp4
Started extracting frames from /content/video2.mp4
Ended extracting frames from /content/video2.mp4
Started extracting frames from /content/video3.mp4
Ended extracting frames from /content/video3.mp4
Started extracting frames from /content/video4.mp4
Ended extracting frames from /content/video4.mp4
Started extracting frames from /content/video5.mp4
Ended extracting frames from /content/video5.mp4
Started extracting frames from /content/video6.mp4
Ended extracting frames from /content/video6.mp4
Started extracting frames from /content/video7.mp4
Ended extracting frames from /content/

In [38]:
print("Copying the data to Google drive directory")
!cp /content/temp1/* /content/drive/My\ Drive/computer_vision/capstone_dataset/inputs/interior_input_images/

Copying the data to Google drive directory


In [39]:
!ls /content/drive/My\ Drive/computer_vision/capstone_dataset/inputs/interior_input_images/ | wc -l

7037


In [40]:
# images = glob.glob("/content/sample_data/temp/*.jpg")
# len(images)

In [41]:
# from IPython.display import Image
# Image(filename='/content/sample_data/temp/img022.jpg')